In [1]:
import os
from tqdm import tqdm
import base64

In [14]:
images_folder = 'images'
image_files = sorted([f for f in os.listdir(images_folder) if f.endswith('.jpg') or f.endswith('.png')], key=lambda x: int(os.path.splitext(x)[0]))

In [3]:

# Função para codificar a imagem em base64
def encode_image_to_base64(image_path):
    with open(image_path, 'rb') as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [4]:
from PIL import Image
import numpy as np

widths = []
heights = []

for image_file in image_files:
    with Image.open(os.path.join(images_folder, image_file)) as img:
        widths.append(img.width)
        heights.append(img.height)

mean_width = np.mean(widths)
mean_height = np.mean(heights)

print(f'Média da largura: {mean_width}')
print(f'Média da altura: {mean_height}')

Média da largura: 835.3571428571429
Média da altura: 1514.357142857143


# Gemini

In [19]:
import google.generativeai as genai

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  return file

In [20]:
files = [
  upload_to_gemini(f"{images_folder}/{image}", mime_type="image/jpeg") for image in tqdm(image_files)
]

100%|██████████| 28/28 [01:33<00:00,  3.33s/it]


In [23]:
model_name = "gemini-2.0-flash-thinking-exp-1219"
logs_folder = "logs/gemini"
prompt = """Extraia as informações dessa imagem, 
            Responda apenas com um json, com os campos:
            is_doc, que é um booleano que indica se o foco da imagem é um documento,
            tipo_imagem, que é uma string que indica o tipo da imagem,
            info, que é um dicionário com as informações estruturadas extraídas da imagem.
            """

# Create the model
generation_config = {
  "temperature": 0,
  "top_p": 0.95,
  "top_k": 1,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name=model_name,
  generation_config=generation_config,
)

for image in tqdm(image_files):
    log_file = os.path.join(logs_folder, os.path.splitext(image)[0] + '.log')

    if os.path.exists(log_file):
        with open(log_file, 'r') as f:
            if f.read().strip().endswith('EOF'):
                continue

    file_index = int(os.path.splitext(image)[0])
    chat_session = model.start_chat(
    history=[
        {
        "role": "user",
        "parts": [
            files[file_index],
        ],
        },
    ]
    )

    response = chat_session.send_message(prompt)
    resultado = response.text

    with open(log_file, 'a') as f:
        if os.path.getsize(log_file) > 0:   f.write('\n' * 5)
        f.write(f'model_name\n{model_name}\n\nprompt\n{prompt}\n\nresultado\n{resultado}\n\nEOF')


100%|██████████| 28/28 [00:38<00:00,  1.37s/it]


# GPT

In [24]:
from openai import OpenAI

client = OpenAI()

## GPT 4 Mini

In [25]:
model_name = "gpt-4o-mini"
logs_folder = "logs/gpt4_mini"
prompt = """Extraia as informações dessa imagem, 
            Responda apenas com um json, com os campos:
            is_doc, que é um booleano que indica se o foco da imagem é um documento,
            tipo_imagem, que é uma string que indica o tipo da imagem,
            info, que é um dicionário com as informações estruturadas extraídas da imagem.
            """


for image in tqdm(image_files):
    log_file = os.path.join(logs_folder, os.path.splitext(image)[0] + '.log')

    if os.path.exists(log_file):
        with open(log_file, 'r') as f:
            if f.read().strip().endswith('EOF'):
                continue

    # Path to your image
    image_path = f"{images_folder}/{image}"

    # Getting the base64 string
    base64_image = encode_image_to_base64(image_path)

    response = client.chat.completions.create(
        model=model_name,
        temperature=0,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt,
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                    },
                ],
            }
        ],
    )

    
    resultado = response.choices[0].message.content

    with open(log_file, 'a') as f:
        if os.path.getsize(log_file) > 0:   f.write('\n' * 5)
        f.write(f'model_name\n{model_name}\n\nprompt\n{prompt}\n\nresultado\n{resultado}\n\nEOF')


100%|██████████| 28/28 [03:15<00:00,  6.98s/it]


## GPT4

In [26]:
model_name = "gpt-4o"
logs_folder = "logs/gpt4"
prompt = """Extraia as informações dessa imagem, 
            Responda apenas com um json, com os campos:
            is_doc, que é um booleano que indica se o foco da imagem é um documento,
            tipo_imagem, que é uma string que indica o tipo da imagem,
            info, que é um dicionário com as informações estruturadas extraídas da imagem.
            """


for image in tqdm(image_files):
    log_file = os.path.join(logs_folder, os.path.splitext(image)[0] + '.log')

    if os.path.exists(log_file):
        with open(log_file, 'r') as f:
            if f.read().strip().endswith('EOF'):
                continue

    # Path to your image
    image_path = f"{images_folder}/{image}"

    # Getting the base64 string
    base64_image = encode_image_to_base64(image_path)

    response = client.chat.completions.create(
        model=model_name,
        temperature=0,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt,
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                    },
                ],
            }
        ],
    )

    
    resultado = response.choices[0].message.content

    with open(log_file, 'a') as f:
        if os.path.getsize(log_file) > 0:   f.write('\n' * 5)
        f.write(f'model_name\n{model_name}\n\nprompt\n{prompt}\n\nresultado\n{resultado}\n\nEOF')


100%|██████████| 28/28 [02:16<00:00,  4.89s/it]


# Groq


In [6]:
from groq import Groq

client = Groq()

## groq Mini

In [7]:
model_name = "llama-3.2-11b-vision-preview"
logs_folder = "logs/groq_mini"
prompt = """Extraia as informações dessa imagem, 
            Responda apenas com um json, com os campos:
            is_doc, que é um booleano que indica se o foco da imagem é um documento,
            image_type, que é uma string que indica o tipo da imagem,
            info, que é um dicionário com as informações estruturadas extraídas da imagem.
            """



for image in tqdm(image_files):
    log_file = os.path.join(logs_folder, os.path.splitext(image)[0] + '.log')

    if os.path.exists(log_file):
        with open(log_file, 'r') as f:
            if f.read().strip().endswith('EOF'):
                continue


    image_path = f"{images_folder}/{image}"

    # Codificar a imagem em base64
    image_base64 = encode_image_to_base64(image_path)

    # Criar a solicitação de completions
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{image_base64}"
                        },
                    },
                    {"type": "text", "text": prompt},
                ],
            }
        ],
        model=model_name,
        temperature=0,
    )

    resultado = chat_completion.choices[0].message.content

    with open(log_file, 'a') as f:
        if os.path.getsize(log_file) > 0:   f.write('\n' * 5)
        f.write(f'model_name\n{model_name}\n\nprompt\n{prompt}\n\nresultado\n{resultado}\n\nEOF')



100%|██████████| 28/28 [01:31<00:00,  3.25s/it]


## groq Full

In [8]:
model_name = "llama-3.2-90b-vision-preview"
logs_folder = "logs/groq"
prompt = """Extraia as informações dessa imagem, 
            Responda apenas com um json, com os campos:
            is_doc, que é um booleano que indica se o foco da imagem é um documento,
            image_type, que é uma string que indica o tipo da imagem,
            info, que é um dicionário com as informações estruturadas extraídas da imagem.
            """



for image in tqdm(image_files):
    log_file = os.path.join(logs_folder, os.path.splitext(image)[0] + '.log')

    if os.path.exists(log_file):
        with open(log_file, 'r') as f:
            if f.read().strip().endswith('EOF'):
                continue


    image_path = f"{images_folder}/{image}"

    # Codificar a imagem em base64
    image_base64 = encode_image_to_base64(image_path)

    # Criar a solicitação de completions
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{image_base64}"
                        },
                    },
                    {"type": "text", "text": prompt},
                ],
            }
        ],
        model=model_name,
        temperature=0,
    )

    resultado = chat_completion.choices[0].message.content

    with open(log_file, 'a') as f:
        if os.path.getsize(log_file) > 0:   f.write('\n' * 5)
        f.write(f'model_name\n{model_name}\n\nprompt\n{prompt}\n\nresultado\n{resultado}\n\nEOF')

100%|██████████| 28/28 [01:29<00:00,  3.21s/it]


# Claude

In [9]:
import anthropic

client = anthropic.Anthropic()

In [10]:
model_name = "claude-3-5-sonnet-20241022"
logs_folder = "logs/claude"
prompt = """Extraia as informações dessa imagem, 
            Responda apenas com um json, com os campos:
            is_doc, que é um booleano que indica se o foco da imagem é um documento,
            tipo_imagem, que é uma string que indica o tipo da imagem,
            info, que é um dicionário com as informações estruturadas extraídas da imagem.
            """


for image in tqdm(image_files):
    log_file = os.path.join(logs_folder, os.path.splitext(image)[0] + '.log')

    if os.path.exists(log_file):
        with open(log_file, 'r') as f:
            if f.read().strip().endswith('EOF'):
                continue

    # Path to your image
    image_path = f"{images_folder}/{image}"

    image_data = encode_image_to_base64(image_path)
    media_type = "image/jpeg" 


    
    message = client.messages.create(
        model=model_name,
        max_tokens=1024*8,
        temperature=0,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": media_type,
                            "data": image_data,
                        },
                    },
                    {
                        "type": "text",
                        "text": prompt,
                    }
                ],
            }
        ],
    )
    
    resultado = message.content[0].text

    with open(log_file, 'a') as f:
        if os.path.getsize(log_file) > 0:   f.write('\n' * 5)
        f.write(f'model_name\n{model_name}\n\nprompt\n{prompt}\n\nresultado\n{resultado}\n\nEOF')


100%|██████████| 28/28 [00:00<00:00, 717.86it/s]


# Todo https://build.nvidia.com/nvidia/neva-22b

# TODO https://build.nvidia.com/microsoft/phi-3-vision-128k-instruct?snippet_tab=Python

## via API

In [29]:
import requests
import os
api_key = os.getenv("PHI3V_API_KEY")


In [49]:
import requests
import os
def upload_to_nvidia(file_path):
    jwt_token = os.getenv("PHI3V_API_KEY")

    # Define o URL da API e o JWT Token de autenticação
    url = 'https://api.nvcf.nvidia.com/v2/nvcf/assets'


    # Cabeçalhos da requisição
    headers = {
        'Authorization': f'Bearer {jwt_token}',
        'Accept': 'application/json',
        'Content-Type': 'application/json',
    }

    # Corpo da requisição
    data = {
        "contentType": "image/png",
        "description": "MQ_chat_image"
    }

    # Realiza a requisição POST
    response = requests.post(url, headers=headers, json=data)

    # Verifica se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Extrai o assetId e a uploadUrl do corpo da resposta
        response_data = response.json()
        asset_id = response_data.get("assetId")
        upload_url = response_data.get("uploadUrl")
        
        pass
        pass
    else:
        pass
        pass

    # Cabeçalhos da requisição PUT
    headers_put = {
        'Content-Type': 'image/png',
        'x-amz-meta-nvcf-asset-description': 'MQ_chat_image'
    }

    # Realiza a requisição PUT para o upload do arquivo
    with open(file_path, 'rb') as file:
        response_put = requests.put(upload_url, headers=headers_put, data=file)

    # Verifica se o upload foi bem-sucedido
    if response_put.status_code == 200:
        pass
    else:
        pass
        pass
    print(response.json()['uploadUrl'])
    return response.json()['uploadUrl']


In [50]:
import requests
import os
from tqdm import tqdm

invoke_url = "https://ai.api.nvidia.com/v1/vlm/microsoft/phi-3-vision-128k-instruct"
stream = False

headers = {
    "Authorization": f"Bearer {api_key}",
    "Accept": "text/event-stream" if stream else "application/json"
}

model_name = "phi-3-vision-128k-instruct"
logs_folder = "logs/phi3v"

prompt = """Extraia as informações dessa imagem, 
            Responda apenas com um json, com os campos:
            is_doc, que é um booleano que indica se o foco da imagem é um documento,
            tipo_imagem, que é uma string que indica o tipo da imagem,
            info, que é um dicionário com as informações estruturadas extraídas da imagem.
            """



# Crie a pasta de logs, se não existir
os.makedirs(logs_folder, exist_ok=True)

for image in tqdm(image_files):
    # Criar o nome do arquivo de log com base na URL
    log_file = os.path.join(logs_folder, os.path.splitext(image)[0] + '.log')

    # Pular se já processado
    if os.path.exists(log_file):
        with open(log_file, 'r') as f:
            if f.read().strip().endswith('EOF'):
                continue
    
    image_url = upload_to_nvidia(f"{images_folder}/{image}")
    print(image_url)
    # Criar o payload com a URL da imagem
    payload = {
        "messages": [
            {
                "role": "user",
                "content": f'o que há na imagem? <img src="{image_url}" />'
            }
        ],
        "max_tokens": 2048,
        "temperature": 0.01,
        "top_p": 0.70,
        "stream": stream
    }

    # Fazer a requisição POST
    response = requests.post(invoke_url, headers=headers, json=payload)
    
    # Verificar resposta da API
    if response.status_code != 200:
        print(f"Erro ao processar {image}: {response.status_code}")
        print(response.json())
        continue

    try:
        resultado = response.json()['choices'][0]['message']['content']
    except KeyError:
        print(f"Erro no formato da resposta para {image}")
        print(response.json())
        continue

    # Escrever resultado no arquivo de log
    with open(log_file, 'a') as f:
        if os.path.getsize(log_file) > 0:
            f.write('\n' * 5)
        f.write(f'model_name\n{model_name}\n\nprompt\n{prompt}\n\nresultado\n{resultado}\n\nEOF')
    break
print("Processamento concluído.")


  0%|          | 0/28 [00:00<?, ?it/s]

https://nv-gdn-strap-assets-prd.s3-accelerate.amazonaws.com/0ibafc5Ew8_-nYmbYjgVKav656KCJNKbuEmqzxLAJXo/1dedf37a-99b3-4d43-8353-27151e87aa66?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEJP%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJHMEUCIQC5cDBGYz28%2FFbwC0JVrVukEkxqP6dPdS5ExDmpcC4sfQIgA7%2FbS%2BR4L1fLiyYivZZg2vHtFwdj%2B1%2F2VYiJWVLmSV4qmgUIfBACGgwwNTIyNzc1MjgxMjIiDFGSxtcDjFANnXxXTir3BGpeT2FnfUkeV9Xfn7PJhrV1XCKWpGGYZnshmLzYAFezn5Jwi3kZ8TiR4XdfhGXdHezIElLt0CbV2BUBuHfThS6rIAJKfuXsu2cRO5vdH10C9EKTHd1ZRLHEYPeOMKl3DBiB6YxEJEq062in22C4SeslIuJHUPff8qRvAvcF15WHphqyzrDySu%2BZB3QRQz2pwHmDdKX1FUHeNGa%2Bgcu%2B1m0HbMcL0edDHbhumht4zvcxOZsU%2B6pHUsM6eRaJ9wrCx4BMR5a3WhROWoTu6oK9yqCP8KBjCkjHf6eFElcdvgNvoypvziqMMa3PzeHxizAn3A6MCsbycJA%2BoeDl7V%2FTUxk1%2B7V7ItWs2Vqtsy1qtQ77qb%2BRmZkf1kIelxFCh0vFEN1Lt49SKQoF2o5TdAUtXWbqN8WS5HXOf0lr%2FPi%2BzlmU%2BS%2FBmayfSC9mF4NAQEfqSZVBlMpaYpPw2eI8axChs9Bf5qwIPubkDvTTBz9x2rvaZJo68jGbX%2BOdo%2BCvJXZ1vuEcN66ODkAjj6J4Gy8eoJ15b1njpIvcfnoA8gufU%2FVb58mVTpHZVG%2BEPXtYNSRWOmqeP9T0

 11%|█         | 3/28 [00:49<06:52, 16.48s/it]

Processamento concluído.


In [52]:
import requests

# URL da API para listar os ativos
list_assets_url = "https://api.nvcf.nvidia.com/v2/nvcf/assets"

# Substitua com o seu token JWT
jwt_token = os.getenv("PHI3V_API_KEY")

# Cabeçalhos para autenticação
headers = {
    "Authorization": f"Bearer {jwt_token}",
    "Accept": "application/json"
}

# Realizar a requisição GET para listar os ativos
response = requests.get(list_assets_url, headers=headers)

if response.status_code == 200:
    # Se a requisição for bem-sucedida, exiba os ativos
    assets = response.json().get("assets", [])
    print(f"Encontrados {len(assets)} ativos:")
    for asset in assets:

        print(f"ID: {asset.get('assetId')}, URL: {asset.get('uploadUrl')}, Descrição: {asset.get('description')}")
        print(asset.keys())
else:
    print(f"Erro ao listar os ativos: {response.status_code}")
    print(response.text)


Encontrados 77 ativos:
ID: 03b01463-6636-4a3e-9e65-59fda492f746, URL: None, Descrição: None
dict_keys(['assetId'])
ID: 040791a5-792d-4797-9d4c-27482d1d133d, URL: None, Descrição: None
dict_keys(['assetId'])
ID: 06950a09-a84d-46a2-b4ba-f548f2871b4f, URL: None, Descrição: None
dict_keys(['assetId'])
ID: 069fe330-3030-4dce-8c25-2a46fc870a29, URL: None, Descrição: None
dict_keys(['assetId'])
ID: 0912eec2-95a0-481f-98ef-f1c530abfa51, URL: None, Descrição: None
dict_keys(['assetId'])
ID: 09f9d4d5-667b-4572-adc4-ba4d2d1fb99b, URL: None, Descrição: None
dict_keys(['assetId'])
ID: 0a538604-c5ef-4bde-92dc-7af8b458973a, URL: None, Descrição: None
dict_keys(['assetId'])
ID: 11c8928a-1e91-4410-b50e-fd2dfaa09a7b, URL: None, Descrição: None
dict_keys(['assetId'])
ID: 131b4aff-904c-4f82-85b7-065c76055768, URL: None, Descrição: None
dict_keys(['assetId'])
ID: 13ec4c9d-b6dd-4d81-8f24-48cb5c7e63aa, URL: None, Descrição: None
dict_keys(['assetId'])
ID: 148d4021-e36f-4984-88e7-1d966b16e0da, URL: None, Desc

In [53]:
import requests

# URL da API para consultar o ativo específico
asset_id = "eb7f3024-9a5a-4876-ad65-52f3e23ca7e2"
get_asset_url = f"https://api.nvcf.nvidia.com/v2/nvcf/assets/{asset_id}"

# Cabeçalhos para autenticação
headers = {
    "Authorization": f"Bearer {jwt_token}",
    "Accept": "application/json"
}

# Realizar a requisição GET
response = requests.get(get_asset_url, headers=headers)

if response.status_code == 200:
    asset = response.json()
    print(f"Asset ID: {asset.get('assetId')}")
    print(f"URL: {asset.get('uploadUrl')}")
    print(f"Descrição: {asset.get('description')}")
    print(f"Tipo de conteúdo: {asset.get('contentType')}")
else:
    print(f"Erro ao consultar o ativo: {response.status_code}")
    print(response.text)


Asset ID: None
URL: None
Descrição: None
Tipo de conteúdo: None


In [54]:
response.json()

{'asset': {'assetId': 'eb7f3024-9a5a-4876-ad65-52f3e23ca7e2',
  'description': 'MQ_chat_image',
  'contentType': 'image/png',
  'createdAt': '2025-01-08T18:27:13Z'}}

## Local

In [6]:
from PIL import Image 
import requests 
from transformers import AutoModelForCausalLM 
from transformers import AutoProcessor 

model_id = "microsoft/Phi-3-vision-128k-instruct" 

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", trust_remote_code=True, torch_dtype="auto", _attn_implementation='flash_attention_2') # use _attn_implementation='eager' to disable flash attention

processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True) 

messages = [ 
    {"role": "user", "content": "<|image_1|>\nWhat is shown in this image?"}, 
    {"role": "assistant", "content": "The chart displays the percentage of respondents who agree with various statements about their preparedness for meetings. It shows five categories: 'Having clear and pre-defined goals for meetings', 'Knowing where to find the information I need for a meeting', 'Understanding my exact role and responsibilities when I'm invited', 'Having tools to manage admin tasks like note-taking or summarization', and 'Having more focus time to sufficiently prepare for meetings'. Each category has an associated bar indicating the level of agreement, measured on a scale from 0% to 100%."}, 
    {"role": "user", "content": "Provide insightful questions to spark discussion."} 
] 

url = "https://assets-c4akfrf5b4d3f4b7.z01.azurefd.net/assets/2024/04/BMDataViz_661fb89f3845e.png" 
image = Image.open(requests.get(url, stream=True).raw) 

prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = processor(prompt, [image], return_tensors="pt").to("cuda:0") 

generation_args = { 
    "max_new_tokens": 500, 
    "temperature": 0.0, 
    "do_sample": False, 
} 

generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args) 

# remove input tokens 
generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0] 

print(response)


/home/rodeys/.cache/pypoetry/virtualenvs/pandd-imagens-A4lvQ6DR-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ImportError: 
AutoModelForCausalLM requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [5]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


ModuleNotFoundError: No module named 'torch'

# TODO https://docs.api.nvidia.com/nim/reference/multimodal-apis